In [4]:
import ast
import random
import pickle
import json

import pandas as pd
import numpy as np
from numpy.linalg.linalg import LinAlgError
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'  # https://stackoverflow.com/questions/34387893/output-matplotlib-figure-to-svg-with-text-as-text-not-curves
from matplotlib_venn import venn2, venn3, venn3_circles
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
import dateutil
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV
from scipy.stats import chi2

In [2]:
animals_used = [1091, 1093, 1060, 1062, 1074, 1092, 1102, 1076, 1082, 1101]
diabetic = [1076, 1082, 1101]
impaired = [1060, 1062, 1074, 1092, 1102]
normal = [1091, 1093]
bg_dict = {'diabetic': diabetic, 'impaired': impaired, 'normal': normal}
animal_to_tolerance_dict = {1076: 'diabetic', 1082: 'diabetic',1101: 'diabetic',
                            1060: 'impaired', 1062: 'impaired', 1074: 'impaired',
                            1092: 'impaired', 1102: 'impaired', 1091: 'normal', 1093: 'normal',}

obs = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='cleaned')
obs['animal'] = obs['animal'].str.strip('TM').astype('int')
obs['datetime'] = obs['when'].apply(dateutil.parser.parse)
animal_phenotypes = pd.read_excel(r'..\data\metadata\20210421_FBGRBGOGTT_Huishi sample sheet.xlsx', sheet_name='animal phenotypes')
animal_phenotypes['animal'] = animal_phenotypes['animal'].str.strip('TM').astype('int')

fg = pd.read_csv(r'..\data\metadata\combined_metab_lipid_file_grouping.csv', index_col=0)  # fg = file grouping

df = pd.read_csv(r'..\data\processed\metab_lipid_merged_df.csv', index_col=0).T

groups = fg[['combined_col_name', 'animal', 'bg_type', 
             'glucose_tolerance', 'week', 'bg']].set_index('combined_col_name')
feat_cols = df.columns
df = df.join(groups)

In [5]:
df

,m_0,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,...,l_1024,l_1025,l_1026,l_1027,l_1028,animal,bg_type,glucose_tolerance,week,bg
1102_8_RBG,30.624453,29.689560,29.931633,28.978826,24.596731,30.066361,29.242583,29.096030,29.090806,22.659910,...,18.104875,18.821648,17.878105,16.200014,19.311939,1102,RBG,impaired,8,299.500000
1093_9_RBG,30.644187,29.774822,29.942531,29.128997,26.201371,29.807669,29.301146,29.054638,28.743299,22.576266,...,18.387917,19.088279,17.974406,15.915763,18.531923,1093,RBG,normal,9,83.000000
1060_8_FBG,30.560702,29.945500,29.528341,29.839308,17.148994,27.075393,30.003016,28.924166,28.562511,27.506896,...,20.517490,19.164463,20.603688,19.836798,18.993666,1060,FBG,impaired,8,41.000000
1060_8_RBG,30.610134,29.309067,29.827946,28.459047,17.431244,29.295561,29.113441,28.848762,28.995708,22.512901,...,17.921098,18.430542,17.488155,16.202162,17.985491,1060,RBG,impaired,8,225.000000
1060_9_FBG,30.606110,30.082627,29.599444,30.235353,19.169210,27.794790,29.908267,28.987014,28.705425,27.900257,...,20.671826,19.113894,20.788152,20.234283,18.592023,1060,FBG,impaired,9,48.000000
1060_9_RBG,30.996475,29.431175,30.018239,28.625995,26.312933,30.211206,29.212933,29.416956,29.331185,22.829664,...,18.261890,18.854601,17.718258,16.494586,18.466842,1060,RBG,impaired,9,83.000000
1060_10_FBG,29.787913,30.096643,29.253402,30.252208,27.209158,28.375135,29.968325,29.158594,28.380032,27.560226,...,19.568571,17.625169,19.769735,19.279616,16.681522,1060,FBG,impaired,10,52.500000
1060_10_RBG,30.636811,29.270487,29.673409,28.265490,25.956118,29.741927,29.297766,28.951930,28.645279,22.775302,...,17.211559,18.154900,17.016673,15.240250,17.829754,1060,RBG,impaired,10,206.000000
1062_8_FBG,30.572489,30.703658,29.569621,30.641931,21.774699,27.427681,30.349551,28.813080,28.671019,26.901321,...,19.348489,17.364343,19.240306,18.585339,16.313637,1062,FBG,impaired,8,52.333333
1062_8_RBG,30.636650,30.109212,30.189935,29.492769,26.822243,30.156755,29.559987,29.064005,29.247692,23.578358,...,18.996482,18.653201,18.778798,16.471090,19.285159,1062,RBG,impaired,8,256.500000


In [9]:
en = ElasticNet()
en.fit(X=df.filter(regex='m_|l_'), y=df['glucose_tolerance'])

ValueError: could not convert string to float: 'impaired'

In [ ]:
sm_en = sm.OLS()